In [1]:
import os
import sys
project_dir = os.path.join(os.pardir, os.pardir)
sys.path.append(project_dir)

import dotenv
dotenv_path = os.path.join(project_dir, '.env')
dotenv.load_dotenv(dotenv_path)

import time
import numpy as np
import pandas as pd
import geopandas as gpd
from sqlalchemy import MetaData, create_engine, extract, select
from sqlalchemy.engine.url import URL
from sqlalchemy.sql import or_
import datetime
from pytz import timezone
from shapely.geometry import Point

from src.data.processing_func import (get_direction, connect_database, extract_geo_sections)
from src.data.load_func import (extract_jps,
                                transf_flow_features,
                                transf_flow_labels)

pd.options.display.max_columns = 30

In [2]:
#Connection and initial setup
meta = connect_database()

date_begin = datetime.date(day=1, month=10, year=2017)
date_end = datetime.date(day=25, month=10, year=2017)

periods = [(7,9), (17,19)]

df_jps = extract_jps(meta, date_begin, date_end, periods=periods, weekends=True, summary=False)
df_jps.sample(5)

,MgrcDateStart,JpsId,SctnId,JamId,JamIndLevelOfTraffic,JamQtdLengthMeters,JamSpdMetersPerSecond,JamTimeDelayInSeconds,JamDscCoordinatesLonLat,JamSpdKmPerHour,LonDirection,LatDirection,date,hour,minute,period,minute_bin
13908,2017-10-03 18:51:00-03:00,226299.0,14009.0,481770.0,4.0,285.0,1.541667,157.0,"[{'x': -48.846533, 'y': -26.303657}, {'x': -48...",5.55,West,North,2017-10-03,18,51,1,45 a 59
6270,2017-10-02 18:31:00-03:00,277696.0,3443.0,479140.0,3.0,1349.0,5.036111,153.0,"[{'x': -48.847522, 'y': -26.337247}, {'x': -48...",18.13,East,South,2017-10-02,18,31,1,30 a 44
152489,2017-10-19 18:36:00-02:00,510550.0,7382.0,542635.0,3.0,587.0,2.469444,167.0,"[{'y': -26.289683, 'x': -48.845373}, {'y': -26...",8.89,West,South,2017-10-19,18,36,1,30 a 44
114838,2017-10-16 18:56:00-02:00,495807.0,15081.0,526635.0,3.0,1389.0,6.061111,123.0,"[{'x': -48.8289, 'y': -26.30954}, {'x': -48.82...",21.82,East,North,2017-10-16,18,56,1,45 a 59
6463,2017-10-02 18:31:00-03:00,206212.0,6098.0,479203.0,3.0,580.0,4.763889,65.0,"[{'x': -48.819743, 'y': -26.254253}, {'x': -48...",17.15,West,North,2017-10-02,18,31,1,30 a 44


In [3]:
#Filter holidays from list of holidays in Brazil, from ANBIMA
holidays = pd.read_excel(project_dir + "/data/external/feriados_nacionais.xls", skip_footer=9)
holidays["Data"] = holidays["Data"].dt.date
holiday_list = holidays["Data"].tolist()
df_jps = df_jps[~df_jps["date"].isin(holiday_list)]

#Aggregate traffic in slots of 15 minutes
jps_per_timeslot = df_jps.groupby(["SctnId", "hour",
                                 "minute_bin", "LonDirection","LatDirection"]) \
                                      .agg({"JpsId": ['count'],
                                           "JamQtdLengthMeters": ["mean"],
                                           "JamSpdKmPerHour": ["mean"],
                                           "JamTimeDelayInSeconds": ["mean"],
                                           "JamIndLevelOfTraffic": ["mean"],
                                           "period": ["max"],
                                           })
jps_per_timeslot.columns = [''.join(col_name).strip() for col_name in jps_per_timeslot.columns.values]
jps_per_timeslot.sample(5)

,,,,,JpsIdcount,JamQtdLengthMetersmean,JamSpdKmPerHourmean,JamTimeDelayInSecondsmean,JamIndLevelOfTrafficmean,periodmax
SctnId,hour,minute_bin,LonDirection,LatDirection,,,,,,
3071.0,7,45 a 59,West,South,8,298.625000,6.675000,111.500000,4.000000,-1
6097.0,8,45 a 59,West,North,1,977.000000,20.420000,85.000000,2.000000,-1
7404.0,18,15 a 29,East,South,34,775.029412,11.107059,191.705882,2.970588,1
15178.0,7,0 a 14,East,North,1,794.000000,22.560000,69.000000,3.000000,-1
5121.0,17,0 a 14,West,South,1,253.000000,7.270000,93.000000,3.000000,1


In [4]:
#Merge dataset with total number of waze signals, for each bin
jps_per_timeslot.reset_index(level=["SctnId", "LonDirection","LatDirection"], inplace=True)

wazesignals_per_timeslot = df_jps.groupby(["hour", "minute_bin"]).agg({"MgrcDateStart": [pd.Series.nunique]})
wazesignals_per_timeslot.columns = ["".join(x) for x in wazesignals_per_timeslot.columns.ravel()]
jps_per_timeslot = jps_per_timeslot.join(wazesignals_per_timeslot, how="outer")

jps_per_timeslot["traffic_prob"] = jps_per_timeslot["JpsIdcount"]/jps_per_timeslot["MgrcDateStartnunique"]

jps_per_timeslot.sample(5)

,,SctnId,LonDirection,LatDirection,JpsIdcount,JamQtdLengthMetersmean,JamSpdKmPerHourmean,JamTimeDelayInSecondsmean,JamIndLevelOfTrafficmean,periodmax,MgrcDateStartnunique,traffic_prob
hour,minute_bin,,,,,,,,,,,
18,45 a 59,13713.0,East,South,3,537.0,12.553333,74.666667,3.000000,1,69,0.043478
8,15 a 29,3743.0,West,North,6,520.0,11.476667,114.166667,3.166667,-1,69,0.086957
18,15 a 29,11815.0,West,North,2,568.5,14.320000,77.500000,2.500000,1,69,0.028986
7,30 a 44,2165.0,East,South,5,472.0,10.286000,130.000000,3.400000,-1,69,0.072464
17,0 a 14,15732.0,East,South,5,1317.8,17.710000,179.600000,2.600000,1,69,0.072464


In [5]:
#Merge dataset with official street sections from the Municipality
geo_sections = extract_geo_sections(meta, buffer=10)
jps_per_timeslot.reset_index(inplace=True)
geo_jps_per_timeslot = geo_sections.merge(jps_per_timeslot, how="inner", on="SctnId")
geo_jps_per_timeslot.set_index(["SctnId", "SctnDscNome", "LonDirection","LatDirection", "hour", "minute_bin"], inplace=True)

columns = {"MgrcDateStartnunique": "Total de sinais do Waze",
         "JpsIdcount": "Engarrafamentos registrados",
         "traffic_prob":"traffic_prob",
         "JamSpdKmPerHourmean": "Velocidade Média (km/h)",
         "JamQtdLengthMetersmean": "Fila média (m)",
         "JamTimeDelayInSecondsmean": "Atraso médio (s)",
         "JamIndLevelOfTrafficmean": "Nível médio de congestionamento (0 a 5)",
         "periodmax": "period",
        }
geo_jps_per_timeslot.rename(columns=columns, inplace=True)
col_list = [col for col in columns.values()]
col_list.append("section_LineString")
geo_jps_per_timeslot = geo_jps_per_timeslot[col_list]
geo_jps_per_timeslot.sample(5)

,,,,,,Total de sinais do Waze,Engarrafamentos registrados,traffic_prob,Velocidade Média (km/h),Fila média (m),Atraso médio (s),Nível médio de congestionamento (0 a 5),period,section_LineString
SctnId,SctnDscNome,LonDirection,LatDirection,hour,minute_bin,,,,,,,,,
5321,OTTO NASS,West,South,8,15 a 29,69,1,0.014493,6.5800,227.00,85.00,3.00,-1,"POLYGON ((-48.851433290526 -26.26074663231489,..."
14709,ADOLFO DA VEIGA,West,North,18,45 a 59,69,2,0.028986,17.3750,882.00,101.50,2.50,1,POLYGON ((-48.82888350659901 -26.3676618853761...
15908,PREFEITO HELMUT FALLGATTER,West,South,7,30 a 44,69,4,0.057971,11.5375,1405.25,309.75,3.25,-1,POLYGON ((-48.8189479701395 -26.29593131760885...
9659,BIGUACU,West,North,18,30 a 44,69,2,0.028986,5.2200,220.00,110.00,3.00,1,POLYGON ((-48.83886071808701 -26.2933072121553...
6619,AVENIDA DOUTOR ALBANO SCHULZ,East,South,17,0 a 14,69,2,0.028986,11.5650,653.00,139.00,3.00,1,POLYGON ((-48.84302063675746 -26.2951634743250...


In [6]:
#Show probability and criticity indicators only for sections of interest
if 'sections_interest' not in globals():
    sections_interest = pd.read_csv(project_dir + "/data/external/vias_estudo.csv", index_col=0, decimal=',')
    
sections_interest.columns = sections_interest.columns.str.strip() 
sections_interest["geometry"] = sections_interest.apply(
                                    lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
crs = geo_jps_per_timeslot.crs
geo_sections_interest = gpd.GeoDataFrame(sections_interest, crs=crs, geometry="geometry")
prob_matrix = gpd.sjoin(geo_sections_interest, geo_jps_per_timeslot, how="left", op="within")
prob_matrix.sample(5)

,Latitude,Longitude,Endereço,Sentido,geometry,SctnId,SctnDscNome,LonDirection,LatDirection,hour,minute_bin,Total de sinais do Waze,Engarrafamentos registrados,traffic_prob,Velocidade Média (km/h),Fila média (m),Atraso médio (s),Nível médio de congestionamento (0 a 5),period
42,-26.32169,-48.83925,RUA FLORIANÓPOLIS,N/S,POINT (-48.83925 -26.32169),13990.0,FLORIANOPOLIS,East,South,17.0,30 a 44,69.0,37.0,0.536232,9.550811,526.243243,147.243243,3.162162,1.0
25,-26.30549,-48.84107,AV. CEL. PROCÓPIO GOMES,N/S,POINT (-48.84107 -26.30549),6271.0,AVENIDA DOUTOR PAULO MEDEIROS,East,South,8.0,45 a 59,71.0,1.0,0.014085,9.260000,444.000000,111.000000,3.000000,-1.0
43,-26.31261,-26.31261,AV. GETÚLIO VARGAS,N/S,POINT (-26.31261 -26.31261),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,-26.29660,-48.84255,AV. DR. ALBANO SCHULZ,N/S,POINT (-48.84255 -26.2966),665.0,AVENIDA DOUTOR ALBANO SCHULZ,West,North,17.0,0 a 14,69.0,4.0,0.057971,16.205000,685.000000,96.250000,2.750000,1.0
16,-26.29591,-48.84206,AV. HERMANN AUGUST LEPPER,S/N,POINT (-48.84206 -26.29591),13245.0,AVENIDA HERMANN AUGUST LEPPER,West,North,17.0,45 a 59,69.0,21.0,0.304348,11.864286,640.857143,151.238095,2.571429,1.0


In [7]:
prob_matrix["notraffic_prob"] = 1 - prob_matrix["traffic_prob"]
prob_matrix["weighted_Velocidade Média (km/h)"] = prob_matrix["traffic_prob"]*prob_matrix["Velocidade Média (km/h)"]
prob_matrix["weighted_Fila média (m)"] = prob_matrix["traffic_prob"]*prob_matrix["Fila média (m)"]
prob_matrix["weighted_Atraso médio (s)"] = prob_matrix["traffic_prob"]*prob_matrix["Atraso médio (s)"]
prob_matrix["weighted_Nível médio de congestionamento (0 a 5)"] = prob_matrix["traffic_prob"]*prob_matrix["Nível médio de congestionamento (0 a 5)"]

traffic_indicators = prob_matrix.groupby(["SctnId", "SctnDscNome",
                                          "Longitude", "Latitude",
                                          "LonDirection", "LatDirection",
                                          "period"]).agg({'notraffic_prob': np.prod,
                                                          'traffic_prob': np.sum,
                                                           "weighted_Velocidade Média (km/h)": np.sum,
                                                           "weighted_Fila média (m)": np.sum,
                                                           "weighted_Atraso médio (s)": np.sum,
                                                           "weighted_Nível médio de congestionamento (0 a 5)": np.sum}) 


traffic_indicators["Probabilidade de Trânsito"] = 1 - traffic_indicators["notraffic_prob"]
traffic_indicators["Velocidade Média (km/h)"] = traffic_indicators["weighted_Velocidade Média (km/h)"] / traffic_indicators["traffic_prob"]
traffic_indicators["Fila média (m)"] = traffic_indicators["weighted_Fila média (m)"] / traffic_indicators["traffic_prob"]
traffic_indicators["Atraso médio (s)"] = traffic_indicators["weighted_Atraso médio (s)"] / traffic_indicators["traffic_prob"]
traffic_indicators["Nível médio de congestionamento (0 a 5)"] = traffic_indicators["weighted_Nível médio de congestionamento (0 a 5)"] / traffic_indicators["traffic_prob"]

traffic_indicators = traffic_indicators[["Probabilidade de Trânsito",
     "Velocidade Média (km/h)",
     "Fila média (m)",
     "Atraso médio (s)",
     "Nível médio de congestionamento (0 a 5)"
     ]
  ]

traffic_indicators.sort_values(by="Probabilidade de Trânsito", ascending=False).head(20)

Probabilidade de Trânsito  \
SctnId  SctnDscNome                        Longitude Latitude  LonDirection LatDirection period                              
13990.0 FLORIANOPOLIS                      -48.83925 -26.32169 East         South         1.0                     0.999042   
                                           -48.83920 -26.32173 East         South         1.0                     0.999042   
7480.0  ITAIOPOLIS                         -48.84354 -26.28744 East         South         1.0                     0.996085   
5515.0  NOVE DE MARCO                      -48.84205 -26.30177 East         South         1.0                     0.990553   
8207.0  GETULIO VARGAS                     -48.84529 -26.31577 East         South         1.0                     0.986876   
5438.0  DOUTOR PLACIDO OLIMPIO DE OLIVEIRA -48.84112 -26.31110 East         North         1.0                     0.981805   
2717.0  DOUTOR PLACIDO OLIMPIO DE OLIVEIRA -48.84040 -26.31108 East         North         1.0                     0.946810   
11433.0 NOVE DE MARCO                      -48.84205 -26.30177 East         South         1.0                     0.934232   
5515.0  NOVE DE MARCO                      -48.84205 -26.30177 East         South        -1.0                     0.908037   
13245.0 AVENIDA HERMANN AUGUST LEPPER      -48.84206 -26.29591 West         North         1.0                     0.891120   
5108.0  CEL. PROCOPIO GOMES                -48.84054 -26.31718 East         South         1.0                     0.889033   
15395.0 CEL. PROCOPIO GOMES                -48.84054 -26.31718 East         South         1.0                     0.889033   
6266.0  PRINCESA IZABEL                    -48.84276 -26.29994 West         North         1.0                     0.872726   
11433.0 NOVE DE MARCO                      -48.84205 -26.30177 East         South        -1.0                     0.861809   
13244.0 AVENIDA HERMANN AUGUST LEPPER      -48.84182 -26.30027 West         North         1.0                     0.854279   
5230.0  DONA FRANCISCA                     -48.84295 -26.29607 East         North         1.0                     0.784781   
13989.0 INACIO BASTOS                      -48.84092 -26.31619 East         North         1.0                     0.743447   
6271.0  AVENIDA DOUTOR PAULO MEDEIROS      -48.84107 -26.30549 East         South         1.0                     0.732974   
7480.0  ITAIOPOLIS                         -48.84354 -26.28744 West         North         1.0                     0.732407   
10629.0 AVENIDA JOSE VIEIRA                -48.84377 -26.28443 East         South        -1.0                     0.725563   

                                                                                                 Velocidade Média (km/h)  \
SctnId  SctnDscNome                        Longitude Latitude  LonDirection LatDirection period                            
13990.0 FLORIANOPOLIS                      -48.83925 -26.32169 East         South         1.0                   9.502762   
                                           -48.83920 -26.32173 East         South         1.0                   9.502762   
7480.0  ITAIOPOLIS                         -48.84354 -26.28744 East         South         1.0                   9.082803   
5515.0  NOVE DE MARCO                      -48.84205 -26.30177 East         South         1.0                   9.177833   
8207.0  GETULIO VARGAS                     -48.84529 -26.31577 East         South         1.0                  10.895223   
5438.0  DOUTOR PLACIDO OLIMPIO DE OLIVEIRA -48.84112 -26.31110 East         North         1.0                   6.776986   
2717.0  DOUTOR PLACIDO OLIMPIO DE OLIVEIRA -48.84040 -26.31108 East         North         1.0                   6.870062   
11433.0 NOVE DE MARCO                      -48.84205 -26.30177 East         South         1.0                   8.860774   
5515.0  NOVE DE MARCO                      -48.84205 -26.30177 East         South   

In [9]:
#Save as csv
traffic_indicators.to_csv(project_dir + "/data/processed/traffic_indicators.csv")